# GDA on histopathological images

we will apply GDA

In [60]:
import numpy as np

In [61]:
import os
from PIL import Image

# Paths to train and test folders
train_folder = "data/histopathologic-cancer-detection/train"
test_folder = "data/histopathologic-cancer-detection/test"

# Load CSV data for train labels
import csv
dico_labels = csv.DictReader(open("data/histopathologic-cancer-detection/train_labels.csv", 'r'))
dicos = [elt for elt in dico_labels]
labels = np.array([elt["label"] for elt in dicos])
train_ids = [elt["id"] for elt in dicos]  # IDs corresponding to training set

# Initialize lists to hold images
train_images = []
test_images = []

# Open train images
for img_id in train_ids:
    img_path = os.path.join(train_folder, f"{img_id}.tif")
    if os.path.exists(img_path):
        try:
            img = np.array(Image.open(img_path))
            length = img.shape[0]
            start,stop = length // 2 - 4, length // 2 + 4
            img_cropped = img[start:stop, start:stop, :]
            train_images.append(img_cropped)
        except Exception as e:
            print(f"Error loading image {img_id} from train folder: {e}")


print(f"Loaded {len(train_images)} train images.")


Loaded 220025 train images.


In [62]:
x = np.array(train_images)

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    x, labels, test_size=0.2, random_state=42
)


In [70]:
from GDA import GDA
model = GDA(n_components=2)
up_to = 50
X_flatten = X_train.reshape(X_train.shape[0], -1)
model.fit(2* X_flatten[:up_to] / 255 -1, y_train[:up_to])


ValueError: array must not contain infs or NaNs